In [1]:
from sklearn.datasets import make_blobs
import pandas as pdm
import numpy as np
import math
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import os
import yaml
import pandas as pd
import ast


In [2]:
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute

In [3]:
# MEMO: top100 games
with open("C:\\Users\weejw\Documents\steamRecommendation\leanring\id_list.txt","r") as file:
    id_list = file.readlines()

app_ids = ast.literal_eval(id_list[0])

# date get from database

In [4]:
with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)
try:
    db_function = db_execute(db_auth['local_db']["host"], 
                             db_auth['local_db']["port"],
                             db_auth['local_db']["user"],
                             db_auth['local_db']["db"], 
                             db_auth['local_db']["passwd"])

    #sql = "SELECT * FROM game_info"
    # MEMO: TOP 100 games
    game_info_sql = f"SELECT * FROM game_info where steam_appid in {str(tuple(app_ids))}"
    review_sql = f"SELECT * FROM review where steam_appid in {str(tuple(app_ids))}"
    
    game_info_rows = db_function.execute_select(game_info_sql)
    review_rows = db_function.execute_select(review_sql)
    
except:
    raise
    


In [5]:
len(game_info_rows), len(review_rows)

(95, 5649)

# game info 정제

In [6]:
game_info_cols = ['id', 'steam_appid', 'name', 'required_age', 'is_free', 'detailed_description', 'about_the_game',
       'short_description', 'developers', 'publishers', 'initial_price', 'final_price', 'discount_percent',
       'platforms', 'release_date', 'supported_languages', 'categories', 'genres', 'recommendations', 'categories_id',
       'genres_id']

In [7]:
_game_info = pd.DataFrame(game_info_rows, columns=game_info_cols)

In [8]:
need_cols=['steam_appid','name','recommendations', 'categories_id', 'genres_id', 'mac', 'linux','windows', 'initial_price', 'final_price', 'discount_percent', 'supported_languages']

In [9]:
_game_info.columns

Index(['id', 'steam_appid', 'name', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'developers', 'publishers', 'initial_price', 'final_price',
       'discount_percent', 'platforms', 'release_date', 'supported_languages',
       'categories', 'genres', 'recommendations', 'categories_id',
       'genres_id'],
      dtype='object')

In [10]:
game_info = _game_info.fillna(0)

In [11]:
game_info['windows']=game_info['platforms'].apply(lambda x:1 if 'windows' in x else 0)
game_info['linux']=game_info['platforms'].apply(lambda x:1 if 'linux' in x else 0)
game_info['mac']=game_info['platforms'].apply(lambda x:1 if 'mac' in x else 0)

In [12]:
game_info['initial_price'] = game_info['initial_price'].apply(lambda x:int(x*0.01))
game_info['final_price'] = game_info['final_price'].apply(lambda x:int(x*0.01))
game_info['recommendations'] = game_info['recommendations'].apply(lambda x:int(x))

In [13]:
game_info = game_info[need_cols]

# review 정제

In [14]:
review_cols = ['id', 'steam_appid', 'steamid', 'num_games_owned', 'playtime_last_two_weeks', 'playtime_at_review',
                             'last_played', 'votes_up', 'votes_funny', 'received_for_free', 'written_during_early_access',
                             'steam_purchase', 'num_reviews', 'playtime_forever', 'review_type', 'review']

In [15]:
_reviews = pd.DataFrame(review_rows, columns=review_cols)

In [16]:
_reviews

,id,steam_appid,steamid,num_games_owned,playtime_last_two_weeks,playtime_at_review,last_played,votes_up,votes_funny,received_for_free,written_during_early_access,steam_purchase,num_reviews,playtime_forever,review_type,review
0,16166,10,76561197967907928,125,0,6731,2018-11-25 20:56:35,2,0,0,0,1,27,8034,positive,말이 필요없음
1,16184,10,76561197973786146,86,0,10,2021-09-04 15:31:22,0,0,0,0,1,7,16,positive,This is where Valve has created the seed of ST...
2,16186,10,76561197974087280,40,0,20265,2014-09-08 23:51:44,0,0,0,0,1,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,16222,10,76561197985593224,94,0,9,2020-06-29 09:23:47,0,0,0,0,1,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,18713,10,76561197995131409,169,0,35,2020-02-14 20:56:10,0,0,0,0,1,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,16912,1468810,76561198031355345,34,1910,8610,2021-12-12 14:57:31,29,1,0,1,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,16914,1468810,76561198036523483,77,0,3510,2021-09-15 13:59:24,1,0,0,1,1,9,4466,positive,GOOD
5646,19498,1468810,76561198095212841,399,0,970,2021-01-31 04:26:58,6,4,0,1,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,19497,1468810,76561198203238402,183,0,29,2021-03-30 18:49:41,7,0,0,1,1,242,29,negative,add korean or english language so i can actua...


In [17]:
need_cols=['steam_appid', 'num_games_owned', 'playtime_last_two_weeks', 'playtime_at_review', 'votes_up', 'received_for_free', 'written_during_early_access', 'num_reviews', 
          'playtime_forever', 'review_type', 'review']

In [18]:
reviews = _reviews[need_cols]

In [19]:
reviews

,steam_appid,num_games_owned,playtime_last_two_weeks,playtime_at_review,votes_up,received_for_free,written_during_early_access,num_reviews,playtime_forever,review_type,review
0,10,125,0,6731,2,0,0,27,8034,positive,말이 필요없음
1,10,86,0,10,0,0,0,7,16,positive,This is where Valve has created the seed of ST...
2,10,40,0,20265,0,0,0,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,10,94,0,9,0,0,0,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,10,169,0,35,0,0,0,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...
5644,1468810,34,1910,8610,29,0,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,1468810,77,0,3510,1,0,1,9,4466,positive,GOOD
5646,1468810,399,0,970,6,0,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,1468810,183,0,29,7,0,1,242,29,negative,add korean or english language so i can actua...


# Merga

In [20]:
all_data = pd.merge(game_info, reviews)

In [21]:
all_data

,steam_appid,name,recommendations,categories_id,genres_id,mac,linux,windows,initial_price,final_price,...,num_games_owned,playtime_last_two_weeks,playtime_at_review,votes_up,received_for_free,written_during_early_access,num_reviews,playtime_forever,review_type,review
0,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,125,0,6731,2,0,0,27,8034,positive,말이 필요없음
1,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,86,0,10,0,0,0,7,16,positive,This is where Valve has created the seed of ST...
2,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,40,0,20265,0,0,0,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,94,0,9,0,0,0,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,169,0,35,0,0,0,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,34,1910,8610,29,0,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,77,0,3510,1,0,1,9,4466,positive,GOOD
5646,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,399,0,970,6,0,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,183,0,29,7,0,1,242,29,negative,add korean or english language so i can actua...


In [42]:
all_data.votes_up.max()

863

In [43]:
all_data.votes_up.mean()

7.303770578863515

In [22]:
all_data.columns

Index(['steam_appid', 'name', 'recommendations', 'categories_id', 'genres_id',
       'mac', 'linux', 'windows', 'initial_price', 'final_price',
       'discount_percent', 'supported_languages', 'num_games_owned',
       'playtime_last_two_weeks', 'playtime_at_review', 'votes_up',
       'received_for_free', 'written_during_early_access', 'num_reviews',
       'playtime_forever', 'review_type', 'review'],
      dtype='object')

# 1. 리뷰 수와 리뷰 찬반율과 추천 수 세가지를 이용해서

In [91]:
rate_table = pd.pivot_table(all_data[['name','review_type','review']], index='name', columns=['review_type'], aggfunc = ['count'])

In [92]:
rate_table = rate_table.droplevel(level=[0,1], axis=1)

In [93]:
rate_table.fillna(0, inplace=True)

In [94]:
rate_table['positive_rate'] = rate_table['positive'] / (rate_table['positive']+rate_table['negative'])

In [95]:
rate_table = rate_table.sort_values(by='positive_rate')

In [96]:
tmp_table = game_info[['name','recommendations']].set_index('name')

In [97]:
rate_table['recommendations'] = tmp_table['recommendations']

In [98]:
rate_table.sort_values(by=['positive_rate', 'recommendations'], ascending=False)[:10]

review_type,negative,positive,positive_rate,recommendations
name,,,,
Dirty Bomb®,0.0,13.0,1.000000,1348
Clicker Heroes,0.0,4.0,1.000000,804
Apex 레전드™,0.0,1.0,1.000000,694
Black Squad,0.0,1.0,1.000000,0
Euro Truck Simulator 2,2.0,75.0,0.974026,377751
Wallpaper Engine,2.0,37.0,0.948718,409858
Half-Life 2: Lost Coast,8.0,100.0,0.925926,7313
Fallout 4,2.0,23.0,0.920000,165669
Tomb Raider,1.0,11.0,0.916667,124604


# 2. votes_up 추가

In [54]:
new_review_data = all_data[['name','review_type','review','votes_up']]

In [56]:
new_review_data['new_review_count'] = 1 + (0.01 * new_review_data['votes_up'])

C:\Users\weejw\AppData\Local\Temp/ipykernel_23168/3710885618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_review_data['new_review_count'] = 1 + (0.01 * new_review_data['votes_up'])


In [58]:
new_review_data.head()

,name,review_type,review,votes_up,new_review_count
0,Counter-Strike,positive,말이 필요없음,2,1.02
1,Counter-Strike,positive,This is where Valve has created the seed of ST...,0,1.00
2,Counter-Strike,positive,말이 필요 없는 한 시대를 풍미한 갓겜,0,1.00
3,Counter-Strike,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...,0,1.00
4,Counter-Strike,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.,0,1.00


In [61]:
new_rate_table = pd.pivot_table(new_review_data[['name','review_type','new_review_count']], index='name', columns=['review_type'], aggfunc = ['sum'])

In [67]:
new_rate_table = new_rate_table.droplevel(level=[0,1], axis=1)

In [70]:
new_rate_table['positive_rate'] = new_rate_table['positive'] / (new_rate_table['positive']+new_rate_table['negative'])

In [71]:
new_rate_table.fillna(0, inplace=True)

In [72]:
new_rate_table

review_type,negative,positive,positive_rate
name,,,
7 Days to Die,4.24,19.04,0.817869
A Story About My Uncle,13.02,78.31,0.857440
ARK: Survival Evolved,20.16,45.04,0.690798
Among Us,10.98,20.21,0.647964
Apex 레전드™,0.00,1.03,1.000000
...,...,...,...
World of Warships,3.06,9.00,0.746269
Z1 Battle Royale,118.61,111.39,0.484304
Z1 Battle Royale: Test Server,15.23,18.13,0.543465


In [73]:
new_rate_table = new_rate_table.sort_values(by='positive_rate')

In [74]:
tmp_table = game_info[['name','recommendations']].set_index('name')

In [75]:
new_rate_table['recommendations'] = tmp_table['recommendations']

In [76]:
new_rate_table.sort_values(by=['positive_rate', 'recommendations'], ascending=False)[:10]

review_type,negative,positive,positive_rate,recommendations
name,,,,
Dirty Bomb®,0.00,13.25,1.000000,1348
Clicker Heroes,0.00,4.03,1.000000,804
Apex 레전드™,0.00,1.03,1.000000,694
Black Squad,0.00,1.01,1.000000,0
Euro Truck Simulator 2,2.02,80.58,0.975545,377751
Wallpaper Engine,2.03,39.22,0.950788,409858
Half-Life 2: Lost Coast,8.10,100.73,0.925572,7313
Fallout 4,2.29,27.76,0.923794,165669
Tomb Raider,1.01,11.84,0.921401,124604


# 3.playtime

In [99]:
rate_table['steam_appid'] = game_info.set_index('name')['steam_appid']

In [100]:
rate_table

review_type,negative,positive,positive_rate,recommendations,steam_appid
name,,,,,
Street Warriors Online,2.0,0.0,0.000000,319,417910
SMITE®,2.0,0.0,0.000000,1325,386360
NBA 2K20,100.0,1.0,0.009901,45760,1089350
Rocket League®,98.0,1.0,0.010101,405107,252950
The Tiny Bang Story,18.0,1.0,0.052632,3192,96000
...,...,...,...,...,...
Euro Truck Simulator 2,2.0,75.0,0.974026,377751,227300
Dirty Bomb®,0.0,13.0,1.000000,1348,333930
Apex 레전드™,0.0,1.0,1.000000,694,1172470


In [101]:
mean_time=_reviews[['steam_appid','playtime_last_two_weeks','playtime_forever']].groupby('steam_appid').agg('mean')

In [102]:
pd.merge(rate_table.reset_index(), mean_time, on='steam_appid').sort_values(by=['positive_rate','playtime_last_two_weeks','playtime_forever'], ascending=False)[:10]

,name,negative,positive,positive_rate,recommendations,steam_appid,playtime_last_two_weeks,playtime_forever
90,Apex 레전드™,0.0,1.0,1.000000,694,1172470,4268.000000,18939.000000
91,Clicker Heroes,0.0,4.0,1.000000,804,363970,0.000000,28825.000000
89,Dirty Bomb®,0.0,13.0,1.000000,1348,333930,0.000000,5172.615385
92,Black Squad,0.0,1.0,1.000000,0,550650,0.000000,2340.000000
88,Euro Truck Simulator 2,2.0,75.0,0.974026,377751,227300,213.480519,7061.350649
87,Wallpaper Engine,2.0,37.0,0.948718,409858,431960,92.230769,4058.846154
86,Half-Life 2: Lost Coast,8.0,100.0,0.925926,7313,340,0.000000,53.990741
85,Fallout 4,2.0,23.0,0.920000,165669,377160,709.680000,18209.440000
84,Tomb Raider,1.0,11.0,0.916667,124604,203160,245.250000,1355.416667
83,Valheim,7.0,64.0,0.901408,271164,892970,256.042254,6529.000000


In [103]:
pd.merge(rate_table.reset_index(), mean_time, on='steam_appid').sort_values(by=['playtime_forever','playtime_last_two_weeks','positive_rate'], ascending=False)[:10]

,name,negative,positive,positive_rate,recommendations,steam_appid,playtime_last_two_weeks,playtime_forever
30,데스티니 가디언즈,13.0,13.0,0.500000,107412,1085660,877.692308,80428.076923
50,ARK: Survival Evolved,17.0,39.0,0.696429,384279,346110,1355.750000,74138.857143
53,Dota 2,14.0,38.0,0.730769,14399,570,119.769231,64015.057692
48,War Thunder,14.0,32.0,0.695652,1656,236390,173.782609,50289.173913
58,World of Warships,3.0,9.0,0.750000,747,552990,570.333333,47544.000000
5,Team Fortress 2,41.0,3.0,0.068182,12786,440,190.272727,45646.931818
70,Warframe,18.0,86.0,0.826923,2910,230410,116.259615,45377.019231
76,Sid Meier's Civilization® V,3.0,20.0,0.869565,110507,8930,499.000000,42932.130435
32,PUBG: BATTLEGROUNDS,92.0,101.0,0.523316,1548549,578080,607.694301,42808.435233
78,The Elder Scrolls V: Skyrim,1.0,7.0,0.875000,177932,72850,10.875000,42310.375000


In [106]:
res = pd.merge(rate_table.reset_index(), mean_time, on='steam_appid').sort_values(by=['playtime_last_two_weeks','playtime_forever','playtime_last_two_weeks'], ascending=False)[:10]

In [107]:
res

,name,negative,positive,positive_rate,recommendations,steam_appid,playtime_last_two_weeks,playtime_forever
90,Apex 레전드™,0.0,1.0,1.000000,694,1172470,4268.000000,18939.000000
24,New World,61.0,49.0,0.445455,179356,1063730,1531.454545,16398.454545
50,ARK: Survival Evolved,17.0,39.0,0.696429,384279,346110,1355.750000,74138.857143
42,Dead by Daylight,56.0,100.0,0.641026,402889,381210,1137.256410,39863.070513
40,Counter-Strike: Global Offensive,13.0,20.0,0.606061,3237267,730,1012.242424,32772.242424
49,Halo Infinite,7.0,16.0,0.695652,10596,1240440,892.739130,1456.956522
30,데스티니 가디언즈,13.0,13.0,0.500000,107412,1085660,877.692308,80428.076923
60,Rust,5.0,17.0,0.772727,572015,252490,875.000000,29759.272727
62,The Witcher® 3: Wild Hunt,8.0,29.0,0.783784,533212,292030,740.378378,12562.621622
85,Fallout 4,2.0,23.0,0.920000,165669,377160,709.680000,18209.440000


In [109]:
res[['name','playtime_last_two_weeks','playtime_forever']].sort_values(by=['playtime_last_two_weeks','playtime_forever','playtime_last_two_weeks'], ascending=False)[:10]

,name,playtime_last_two_weeks,playtime_forever
90,Apex 레전드™,4268.000000,18939.000000
24,New World,1531.454545,16398.454545
50,ARK: Survival Evolved,1355.750000,74138.857143
42,Dead by Daylight,1137.256410,39863.070513
40,Counter-Strike: Global Offensive,1012.242424,32772.242424
49,Halo Infinite,892.739130,1456.956522
30,데스티니 가디언즈,877.692308,80428.076923
60,Rust,875.000000,29759.272727
62,The Witcher® 3: Wild Hunt,740.378378,12562.621622
85,Fallout 4,709.680000,18209.440000
